# Cleaning Up the code

## Comms Check

**Here is the first check we coded, which checked comms in the loads. Notice that comments have been added and uneeded code has been removed for clarity and responsiveness.**

In [1]:
from re import search
from glob import glob

""" This function will take a Backstop file 
  and find all the comms that are contained
  A count is returned to the base program"""

def countComms(backstop_filename):
    # Open the file for reading
    file_in = open(backstop_filename,'r')
    
    # Counter for the amount of comms in the file
    counter=0

    for line in file_in:
        # regular expression search of the current line 
        # from the file
        ctx = search('TLMSID=.(CTX[A,B]ON)',line)
        if ctx:
            # add to the count
            counter+=1
    # Close that file, private pyle!
    file_in.close()
    
    # return your answer out of the  function to 
    # allow the main program to use it
    return counter

**Let's try the function out and see how it works!**

In [2]:
import sys

# Find all backstop files in the Backstop directory
file = glob("/home/mission/Backstop/MAR1918B/*backstop")

# Check if we have only one. Otherwise, slap their hand, and exit
if not(len(file) > 0 and len(file) <2):
    print("Incorrect Amount of Backstop files. Exiting...")
    sys.exit()
else:
    # Do the things
    backstop_file = file[0]

    counter = countComms(backstop_file)

    if counter < 1:
        print("There are no comms included in this loadset.")
    else:
        print("There are " + str(counter) + " comms in this loadset.")

There are 19 comms in this loadset.


**This seems to work great! Let's move on to the next one.**

## Bias Check

** Once again, the code has been cleaned up and beautfied. The comments were left out for us to do.**

In [12]:
from Chandra.Time import DateTime

# This function will take a backstop file and find all XTZ,XCZ, and SELFMT commands. 
# The FMT commands will be checked to see if they occur within 26 minutes of any XTZ or XCZ.
# If there are no problems, the user is made aware of this.
# If there are problems, the user is shown which format commands can potentially be a problem.
def checkBiasVsFormat(backstop_filename):
    regex_to_match = ["(\d+:\d+:\d+:\d+:\d+.\d+).*TLMSID= (X[T C]Z[0-9 A-F]+)",
                         "(\d+:\d+:\d+:\d+:\d+.\d+).*TLMSID= (CSELFMT\d)"] 
    
    # Open the backstop file for reading
    file_in = open(backstop_filename)
    results_array = []

    for line in file_in:
        match = search(regex_to_match[0],line)
        if not match:
            match = search(regex_to_match[1],line)
        if match:
            temp_object = {
                "timestring": match.group(1),
                "tlmsid":match.group(2),
                "time_seconds":DateTime(match.group(1)).secs
            }

            results_array.append(temp_object)
    file_in.close()
    
    previous_object = {}
    flag=0
    for index,result_instance in enumerate(results_array):
        if not index == 0:
            time = result_instance['time_seconds']
            previous_time = previous_object['time_seconds']

            match=search("SELFMT\d",result_instance['tlmsid'])
            if match:
                match2=search("SELFMT\d",previous_object['tlmsid'])

                if not match2:
                    if  previous_time+(26*60) > time:
                        flag=1
                        output = previous_object['tlmsid']+" "+previous_object['timestring'] +\
                        " | "+ result_instance['tlmsid'] +" " +\
                        result_instance['timestring'] + "|<---THIS NEEDS CHECKED | TIME BETWEEN: "+\
                        str(round((result_instance['time_seconds']-previous_object['time_seconds'])/60,2))+' Minutes'

                        print(output)

        previous_object = result_instance
        
    if not flag:
        print('There is no format commanding during an ACIS bias.')

**Since we already have a Backstop to try this on, let's use it!!!**

In [13]:
checkBiasVsFormat(backstop_file)

There is no format commanding during an ACIS bias.


## Using the Kadi Archive 

**Overview**<br>
The kadi Chandra events database stores a variety of events that are useful in performing analysis or establishing situational awareness (i.e. what was happening around March 2017?). Examples include:
a
* Events in telemetry such as maneuvers, NPM dwells, obsids, mech movements, momentum dumps, orbit events, etc.
* CAPs, DSN passes, dark cals, SCS107, safe modes, bright star hold, etc
* Chandra major events since launch

As shown in the Event definitions section, there are a number of different types of Chandra events that are available within the Kadi archive. Each type of event is essentially a database table with a number of data fields, some of them common to all event types and some of them unique.

http://cxc.harvard.edu/mta/ASPECT/tool_doc/kadi/events.html

In [14]:
from kadi import events

This import will give us all events that kadi contains. Here is a list of events using a help function in kadi itself.

In [15]:
events?

You can also use **tab** to show you a list of events like this

events.<TAB>  # type "events." and then press <TAB>

This also works on the event itself, showing you all the fields that the event contains

In [29]:
events.dumps?

This is just the start. You can also output the events to the screen or a variable using the **table** function

In [28]:
events.dumps.filter(start='2018:035',stop='2018:045').table

start,stop,tstart,tstop,dur
str21,str21,float64,float64,float64
2018:037:01:27:54.576,2018:037:01:30:03.726,634267743.76,634267872.91,129.150007606
2018:039:14:55:54.664,2018:039:14:58:15.089,634489023.848,634489164.273,140.425008297
2018:042:05:22:55.103,2018:042:05:25:24.753,634713844.287,634713993.937,149.650008798


In [25]:
a=events.obsids.filter(start='2018:035',stop='2018:036')
a[0].dur
a[0].tstart


634132067.0649269

In [31]:
from kadi import commands